In [1]:
from glob import glob
from tqdm import tqdm
#from rdkit import Chem
import numpy as np
import pandas as pd
import re
import shutil
import os

import autogluon
from autogluon.common.utils.utils import setup_outputdir
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path
import sklearn
from sklearn.model_selection import train_test_split
import autogluon.core as ag
import pandas as pd
import numpy as np

import pandas as pd
import random
import argparse
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

import glob

/opt/conda/envs/bigcon/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import datetime 
import time     

In [3]:
full_data = pd.read_csv("./dataset/full_data.csv")

In [4]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4768898 entries, 0 to 4768897
Data columns (total 34 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   application_id                           int64  
 1   bank_id                                  int64  
 2   product_id                               int64  
 3   loan_limit                               int64  
 4   loan_rate                                float64
 5   is_applied                               float64
 6   user_id                                  int64  
 7   birth_year                               int64  
 8   gender                                   int64  
 9   credit_score                             int64  
 10  yearly_income                            int64  
 11  income_type                              int64  
 12  company_enter_month                      int64  
 13  employment_type                          int64  
 14  houseown_type     

In [5]:
# le_income_type

le_purpose_type = LabelEncoder()
le_purpose_type = le_purpose_type.fit(full_data["purpose"])
le_purpose_fitted = le_purpose_type.fit_transform(full_data["purpose"])
purpose_inverted = le_purpose_type.inverse_transform(le_purpose_fitted)

print(le_purpose_type.classes_)

['기타' '대환대출' '사업자금' '생활비' '자동차구입' '전월세보증금' '주택구입' '투자']


In [6]:
full_data["purpose"] = le_purpose_fitted

In [7]:
full_data["bank_id"] = full_data["bank_id"].astype('category')
full_data["product_id"] = full_data["product_id"].astype('category')
full_data["loan_limit"] = full_data["loan_limit"].astype('category')
full_data["birth_year"] = full_data["birth_year"].astype('category')

full_data["gender"] = full_data["gender"].astype('category')
full_data["credit_score"] = full_data["credit_score"].astype('category')
full_data["yearly_income"] = full_data["yearly_income"].astype('category')
full_data["income_type"] = full_data["income_type"].astype('category')
full_data["company_enter_month"] = full_data["company_enter_month"].astype('category')

full_data["employment_type"] = full_data["employment_type"].astype('category')
full_data["houseown_type"] = full_data["houseown_type"].astype('category')
full_data["desired_amount"] = full_data["desired_amount"].astype('category')
full_data["existing_loan_cnt"] = full_data["existing_loan_cnt"].astype('category')
full_data["existing_loan_amt"] = full_data["existing_loan_amt"].astype('category')

full_data["credit_score_nan"] = full_data["credit_score_nan"].astype('category')
full_data["gender_birth_nan"] = full_data["gender_birth_nan"].astype('category')
full_data["loan_limit/max_loan_limit_binary"] = full_data["loan_limit/max_loan_limit_binary"].astype('category')
full_data["loan_rate/min_loan_rate_binary"] = full_data["loan_rate/min_loan_rate_binary"].astype('category')

full_data["desired_amount/existing_loan_amt_binary"] = full_data["desired_amount/existing_loan_amt_binary"].astype('category')
full_data["hour"] = full_data["hour"].astype('category')
full_data["dayofweek"] = full_data["dayofweek"].astype('category')
full_data["purpose"] = full_data["purpose"].astype('category')
full_data["existing_loan_cnt_binary"] = full_data["existing_loan_cnt_binary"].astype("category")

In [8]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4768898 entries, 0 to 4768897
Data columns (total 34 columns):
 #   Column                                   Dtype   
---  ------                                   -----   
 0   application_id                           int64   
 1   bank_id                                  category
 2   product_id                               category
 3   loan_limit                               category
 4   loan_rate                                float64 
 5   is_applied                               float64 
 6   user_id                                  int64   
 7   birth_year                               category
 8   gender                                   category
 9   credit_score                             category
 10  yearly_income                            category
 11  income_type                              category
 12  company_enter_month                      category
 13  employment_type                          category
 14  ho

In [9]:
full_data["purpose"].unique()

[3, 1, 2, 7, 0, 5, 6, 4]
Categories (8, int64): [0, 1, 2, 3, 4, 5, 6, 7]

In [10]:
full_data = full_data.drop(["application_id", "user_id"], axis = 1)

### K fold ( Set 5 Fold)

In [12]:
def k_fold_result(data):
    ## Test set setting
    data = data.reset_index().iloc[:, 1:]
    
    train, test = train_test_split(data, test_size=0.15,   random_state=20205289,  stratify=data['is_applied'])

    train = train.reset_index().iloc[:, 1:]
    test = test.reset_index().iloc[:, 1:]

    from sklearn.model_selection import StratifiedKFold
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=20205289)
    for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(train)), y=train['is_applied'])):
        train.loc[val_idx, 'fold'] = fold
        
    return train, test

### Modeling

In [13]:
def modeling(dataset, fold):
    
    dataset, test = k_fold_result(dataset)
    test.to_csv(f"./fold_data/model_all_test.csv")
    
    train = dataset.reset_index().iloc[:, 1:]
    train["is_applied"] = train["is_applied"].astype(int)
    
    for i in range(0, fold):

        val_idx = list(train[train['fold'] == int(i)].index)
        val_df = train[train['fold'] == i].reset_index(drop=True).iloc[:, :-1]
        train_df = train[train['fold'] != i].reset_index(drop=True).iloc[:, :-1]

        labels = 'is_applied'
        problem_types = 'binary'  
        save_path = f'model_test_all/model_all_fold_{i}'  

        excluded_model_types = ['KNN', "NN_TORCH", "FASTAI", "CAT", "Cat"]
        multi_predictor = TabularPredictor(label=labels, problem_type=problem_types, eval_metric="f1", path=save_path)
        multi_predictor.fit(train_df, tuning_data=val_df, excluded_model_types=excluded_model_types, hyperparameters = "light") #['default', 'light', 'very_light', 'toy', 'multimodal', 'interpretable']
        
        train_df.to_csv(f"./fold_data/model_all_fold_{i}.csv")
        val_df.to_csv(f"./fold_data/model_all_fold_{i}.csv")
        
        time.sleep(60)
        print("TIME")
    
    return test
    

In [13]:
#'생활비', '대환대출', '사업자금', '투자', '기타', '전월세보증금', '주택구입', '자동차구입'
test_df = modeling(dataset = full_data, fold = 5)

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "model_test_all/model_all_fold_0/"
AutoGluon Version:  0.5.3b20220913
Python Version:     3.8.13
Operating System:   Linux
Train Data Rows:    3242850
Train Data Columns: 31
Tuning Data Rows:    810713
Tuning Data Columns: 31
Label Column: is_applied
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    70922.49 MB
	Train Data (Original)  Memory Usage: 356.72 MB (0.5% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFea

[1000]	valid_set's binary_logloss: 0.207439	valid_set's f1: 0.573246
[2000]	valid_set's binary_logloss: 0.206093	valid_set's f1: 0.576239
[3000]	valid_set's binary_logloss: 0.205477	valid_set's f1: 0.577552
[4000]	valid_set's binary_logloss: 0.205142	valid_set's f1: 0.578479
[5000]	valid_set's binary_logloss: 0.204929	valid_set's f1: 0.57912
[6000]	valid_set's binary_logloss: 0.204816	valid_set's f1: 0.579587
[7000]	valid_set's binary_logloss: 0.204781	valid_set's f1: 0.579851
[8000]	valid_set's binary_logloss: 0.204756	valid_set's f1: 0.579971
[9000]	valid_set's binary_logloss: 0.204792	valid_set's f1: 0.579777
[10000]	valid_set's binary_logloss: 0.204834	valid_set's f1: 0.579642


	0.5803	 = Validation score   (f1)
	4672.88s	 = Training   runtime
	52.75s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_logloss: 0.206785	valid_set's f1: 0.575709
[2000]	valid_set's binary_logloss: 0.205807	valid_set's f1: 0.577656
[3000]	valid_set's binary_logloss: 0.205305	valid_set's f1: 0.578624
[4000]	valid_set's binary_logloss: 0.205149	valid_set's f1: 0.57882
[5000]	valid_set's binary_logloss: 0.205107	valid_set's f1: 0.579459
[6000]	valid_set's binary_logloss: 0.205084	valid_set's f1: 0.578943


	0.5796	 = Validation score   (f1)
	3049.62s	 = Training   runtime
	22.79s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.5414	 = Validation score   (f1)
	304.49s	 = Training   runtime
	4.59s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.5395	 = Validation score   (f1)
	301.36s	 = Training   runtime
	5.4s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.3389	 = Validation score   (f1)
	160.32s	 = Training   runtime
	4.75s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.3365	 = Validation score   (f1)
	160.77s	 = Training   runtime
	4.6s	 = Validation runtime
Fitting model: XGBoost ...
/opt/conda/envs/bigcon/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
	0.5784	 = Validation score   (f1)
	5031.06s	 = Training   runtime
	19.04s	 = Validation

...

[1000]	valid_set's binary_logloss: 0.207055	valid_set's f1: 0.576035
[2000]	valid_set's binary_logloss: 0.205986	valid_set's f1: 0.578072
[3000]	valid_set's binary_logloss: 0.20545	valid_set's f1: 0.578912
[4000]	valid_set's binary_logloss: 0.205322	valid_set's f1: 0.579498
[5000]	valid_set's binary_logloss: 0.205253	valid_set's f1: 0.580283
[6000]	valid_set's binary_logloss: 0.205178	valid_set's f1: 0.580379
[7000]	valid_set's binary_logloss: 0.205144	valid_set's f1: 0.580612
[8000]	valid_set's binary_logloss: 0.205194	valid_set's f1: 0.580779
[9000]	valid_set's binary_logloss: 0.205243	valid_set's f1: 0.580497
[10000]	valid_set's binary_logloss: 0.205364	valid_set's f1: 0.580855


	0.5812	 = Validation score   (f1)
	4356.77s	 = Training   runtime
	59.61s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.5419	 = Validation score   (f1)
	259.14s	 = Training   runtime
	4.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.541	 = Validation score   (f1)
	257.17s	 = Training   runtime
	4.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.3369	 = Validation score   (f1)
	138.88s	 = Training   runtime
	3.97s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.343	 = Validation score   (f1)
	138.6s	 = Training   runtime
	3.98s	 = Validation runtime
Fitting model: XGBoost ...
	0.5802	 = Validation score   (f1)
	3262.14s	 = Training   runtime
	11.69s	 = Validation runtime
Fitting model: LightGBMLarge ...


[1000]	valid_set's binary_logloss: 0.205471	valid_set's f1: 0.578047
[2000]	valid_set's binary_logloss: 0.204606	valid_set's f1: 0.579837
[3000]	valid_set's binary_logloss: 0.204222	valid_set's f1: 0.581043
[4000]	valid_set's binary_logloss: 0.204042	valid_set's f1: 0.582309
[5000]	valid_set's binary_logloss: 0.204032	valid_set's f1: 0.582271
[6000]	valid_set's binary_logloss: 0.204048	valid_set's f1: 0.582361
[7000]	valid_set's binary_logloss: 0.204128	valid_set's f1: 0.582489
[8000]	valid_set's binary_logloss: 0.204274	valid_set's f1: 0.582953
[9000]	valid_set's binary_logloss: 0.20449	valid_set's f1: 0.583456
[10000]	valid_set's binary_logloss: 0.204814	valid_set's f1: 0.583355


	0.5837	 = Validation score   (f1)
	4476.96s	 = Training   runtime
	79.18s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.5838	 = Validation score   (f1)
	282.34s	 = Training   runtime
	1.24s	 = Validation runtime
AutoGluon training complete, total runtime = 17764.86s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("model_test_all/model_all_fold_4/")


TIME


In [14]:
dataset, test = k_fold_result(full_data)

train = dataset.reset_index().iloc[:, 1:]
train["is_applied"] = train["is_applied"].astype(int)


val_idx_0 = list(train[train['fold'] == 0].index)
val_df_0 = train[train['fold'] == 0].reset_index(drop=True).iloc[:, :-1]
train_df_0 = train[train['fold'] != 0].reset_index(drop=True).iloc[:, :-1]

val_idx_1 = list(train[train['fold'] == 1].index)
val_df_1 = train[train['fold'] == 1].reset_index(drop=True).iloc[:, :-1]
train_df_1 = train[train['fold'] != 1].reset_index(drop=True).iloc[:, :-1]

val_idx_2 = list(train[train['fold'] == 2].index)
val_df_2 = train[train['fold'] == 2].reset_index(drop=True).iloc[:, :-1]
train_df_2 = train[train['fold'] != 2].reset_index(drop=True).iloc[:, :-1]

val_idx_3 = list(train[train['fold'] == 3].index)
val_df_3 = train[train['fold'] == 3].reset_index(drop=True).iloc[:, :-1]
train_df_3 = train[train['fold'] != 3].reset_index(drop=True).iloc[:, :-1]

val_idx_4 = list(train[train['fold'] == 4].index)
val_df_4 = train[train['fold'] == 4].reset_index(drop=True).iloc[:, :-1]
train_df_4 = train[train['fold'] != 4].reset_index(drop=True).iloc[:, :-1]


### Test

In [15]:
reg_01 = TabularPredictor.load("model_test_all/model_all_fold_0")
reg_02 = TabularPredictor.load("model_test_all/model_all_fold_1")
reg_03 = TabularPredictor.load("model_test_all/model_all_fold_2")
reg_04 = TabularPredictor.load("model_test_all/model_all_fold_3")
reg_05 = TabularPredictor.load("model_test_all/model_all_fold_4")


name_01 = reg_01.leaderboard(val_df_0, silent=True)
name_02 = reg_02.leaderboard(val_df_1, silent=True)
name_03 = reg_03.leaderboard(val_df_2, silent=True)
name_04 = reg_04.leaderboard(val_df_3, silent=True)
name_05 = reg_05.leaderboard(val_df_4, silent=True)


/opt/conda/envs/bigcon/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [16]:
print("fold_0 : ", name_01["model"][0])
print("fold_1 : ", name_02["model"][0])
print("fold_2 : ", name_03["model"][0])
print("fold_3 : ", name_04["model"][0])
print("fold_4 : ", name_05["model"][0])

fold_0 :  WeightedEnsemble_L2
fold_1 :  WeightedEnsemble_L2
fold_2 :  WeightedEnsemble_L2
fold_3 :  WeightedEnsemble_L2
fold_4 :  WeightedEnsemble_L2


In [17]:
rag_01_model_name = reg_01._trainer.load_model(name_01["model"][0])
rag_02_model_name = reg_02._trainer.load_model(name_02["model"][0])
rag_03_model_name = reg_03._trainer.load_model(name_03["model"][0])
rag_04_model_name = reg_04._trainer.load_model(name_04["model"][0])
rag_05_model_name = reg_05._trainer.load_model(name_05["model"][0])

In [20]:
dataset, test_df = k_fold_result(full_data)

### Evaluation

In [21]:
rag_01_model_p  = reg_01.predict_proba(test_df, model = rag_01_model_name )
rag_02_model_p  = reg_02.predict_proba(test_df, model = rag_02_model_name )
rag_03_model_p  = reg_03.predict_proba(test_df, model = rag_03_model_name )
rag_04_model_p  = reg_04.predict_proba(test_df, model = rag_04_model_name )
rag_05_model_p  = reg_05.predict_proba(test_df, model = rag_05_model_name )